In [15]:
import VASPsol as vs
import pandas as pd
import os
import numpy as np
import COSMO_TL as ctl
from dask.distributed import Client, LocalCluster, progress


In [2]:
cluster = LocalCluster(n_workers=16, threads_per_worker=1, memory_limit='2GB')
client = Client(cluster)

/home/ericfonseca/miniconda3/envs/venv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37080 instead
  warnings.warn(
2023-03-01 15:30:26,459 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-jhn5xdjo', purging
2023-03-01 15:30:26,469 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-qtybfxfd', purging
2023-03-01 15:30:26,476 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-a0zo9xt0', purging
2023-03-01 15:30:26,477 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-yk660ppg', purging
2023-03-01 15:30:26,478 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-sozq98gm', purging
2023-03-01 15:30:26,479 - distributed.disku

In [16]:
# get all the directories with a folder called VAC in them
run_folders = [x[0] for x in os.walk('.') if 'VAC' in os.walk(x[0]).__next__()[1]]
vasp_data = [client.submit(vs.data, i) for i in run_folders]
progress(vasp_data)

VBox()

In [17]:
vasp_data = client.gather(vasp_data)

In [18]:
df = pd.concat([x.ml_df for x in vasp_data]).reset_index(drop=True)
df

,Solvent,FileHandle,NC_K,SIGMA_K,TAU,Solvation_Energy,Total_Energy,No.,SoluteName,Formula,...,OP,S,HS,OS,SP,SS,TotalArea,error,error_ev,error_frac
0,water,0092ethb,0.00300,0.642036,0.000772,-0.194341,-62.879039,2179,ethylformate,H6C3O2,...,0.0,0.000,0.000,0.0,0.0,0.0,137.183,1.831503,0.079423,-0.691133
1,water,0092ethb,0.00366,0.581300,0.000936,-0.187315,-62.872013,2179,ethylformate,H6C3O2,...,0.0,0.000,0.000,0.0,0.0,0.0,137.183,1.669484,0.072397,-0.629994
2,water,0092ethb,0.00250,0.600000,0.000525,-0.177813,-62.862511,2179,ethylformate,H6C3O2,...,0.0,0.000,0.000,0.0,0.0,0.0,137.183,1.450368,0.062895,-0.547309
3,water,0092ethb,0.00275,0.660000,0.000472,-0.227300,-62.911998,2179,ethylformate,H6C3O2,...,0.0,0.000,0.000,0.0,0.0,0.0,137.183,2.591538,0.112382,-0.977939
4,water,0145pro,0.00300,0.642036,0.000772,-0.218914,-55.248212,2231,allylalcohol,H6C3O1,...,0.0,0.000,0.000,0.0,0.0,0.0,125.517,-0.031843,-0.001381,0.006268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,water,0142die,0.00250,0.600000,0.000525,-0.051419,-77.307156,2228,diethylsulfide,H10C4S1,...,0.0,25.165,0.061,0.0,0.0,0.0,172.471,-0.244278,-0.010593,0.170824
892,water,0100met,0.00366,0.581300,0.000936,-0.125560,-129.028030,2187,methylhexanoate,H14C7O2,...,0.0,0.000,0.000,0.0,0.0,0.0,233.686,0.405414,0.017581,-0.162817
893,water,0100met,0.00250,0.600000,0.000525,-0.138300,-129.040770,2187,methylhexanoate,H14C7O2,...,0.0,0.000,0.000,0.0,0.0,0.0,233.686,0.699198,0.030321,-0.280802
894,water,0010met,0.00366,0.581300,0.000936,0.067036,-73.528551,2101,2-methylpropane,H10C4,...,0.0,0.000,0.000,0.0,0.0,0.0,143.412,0.774150,0.033571,0.333685


In [6]:

# get all the sigma profiles for each row
sig_jobs = [client.submit(ctl.gl.tr_sig, i) for i in df['SoluteName'].unique()]
progress(sig_jobs)

VBox()

In [13]:
sig_jobs = client.gather(sig_jobs)

sigma_profiles = []
areas = []
volumes = []
for i in sig_jobs:
    try:
        sigma_profiles.append(i[0])
        areas.append(i[1][0])
        volumes.append(i[1][1])
    except:
        sigma_profiles.append(np.nan)
        areas.append(np.nan)
        volumes.append(np.nan)
    

In [8]:
sig_df = pd.DataFrame({'SoluteName': df['SoluteName'].unique(),
                          'sigma_solute': sigma_profiles,
                            'area_solute': areas,
                            'volume_solute': volumes})


In [9]:
# repeat this process and append the solvent data to the dataframe
sol_jobs = [client.submit(ctl.gl.tr_sig, i) for i in df['Solvent'].unique()]
progress(sol_jobs)


VBox()

In [10]:
sol_jobs = client.gather(sol_jobs)


In [11]:
sigma_profiles = []
areas = []
volumes = []
for i in sol_jobs:
    try:
        sigma_profiles.append(i[0])
        areas.append(i[1][0])
        volumes.append(i[1][1])
    except:
        sigma_profiles.append(np.nan)
        areas.append(np.nan)
        volumes.append(np.nan)

sol_df = pd.DataFrame({'Solvent': df['Solvent'].unique(),
                            'sigma_solvent': sigma_profiles,
                            'area_solvent': areas,
                            'volume_solvent': volumes})
sol_df

,Solvent,sigma_solvent,area_solvent,volume_solvent
0,water,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.26923,25.73454


In [12]:
# merge sol_df and sig_df into df
df2 = pd.merge(df, sig_df, on='SoluteName')
df2 = pd.merge(df2, sol_df, on='Solvent')
df2

,Solvent,FileHandle,NC_K,SIGMA_K,TAU,Solvation_Energy,Total_Energy,No.,SoluteName,Formula,...,TotalArea,error,error_ev,error_frac,sigma_solute,area_solute,volume_solute,sigma_solvent,area_solvent,volume_solvent
0,water,0092ethb,0.00300,0.642036,0.000772,-0.194341,-62.879039,2179,ethylformate,H6C3O2,...,137.183,1.831503,0.079423,-0.691133,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",113.59918,97.18179,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.26923,25.73454
1,water,0092ethb,0.00366,0.581300,0.000936,-0.187315,-62.872013,2179,ethylformate,H6C3O2,...,137.183,1.669484,0.072397,-0.629994,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",113.59918,97.18179,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.26923,25.73454
2,water,0092ethb,0.00250,0.600000,0.000525,-0.177813,-62.862511,2179,ethylformate,H6C3O2,...,137.183,1.450368,0.062895,-0.547309,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",113.59918,97.18179,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.26923,25.73454
3,water,0092ethb,0.00275,0.660000,0.000472,-0.227300,-62.911998,2179,ethylformate,H6C3O2,...,137.183,2.591538,0.112382,-0.977939,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",113.59918,97.18179,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.26923,25.73454
4,water,0145pro,0.00300,0.642036,0.000772,-0.218914,-55.248212,2231,allylalcohol,H6C3O1,...,125.517,-0.031843,-0.001381,0.006268,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.036...",104.17677,86.86917,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.26923,25.73454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,water,0142die,0.00250,0.600000,0.000525,-0.051419,-77.307156,2228,diethylsulfide,H10C4S1,...,172.471,-0.244278,-0.010593,0.170824,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",142.21552,131.05941,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.26923,25.73454
892,water,0100met,0.00366,0.581300,0.000936,-0.125560,-129.028030,2187,methylhexanoate,H14C7O2,...,233.686,0.405414,0.017581,-0.162817,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",271.43889,271.77145,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.26923,25.73454
893,water,0100met,0.00250,0.600000,0.000525,-0.138300,-129.040770,2187,methylhexanoate,H14C7O2,...,233.686,0.699198,0.030321,-0.280802,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",271.43889,271.77145,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.26923,25.73454
894,water,0010met,0.00366,0.581300,0.000936,0.067036,-73.528551,2101,2-methylpropane,H10C4,...,143.412,0.774150,0.033571,0.333685,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",130.24956,125.33547,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.26923,25.73454
